### 헤더 확인

In [1]:
import pandas as pd

# 파일 경로
file_path = "C:/Users/SAMSUNG/OneDrive/바탕 화면/workspace/미드프로젝트/데이터 분석/데이터 전처리/음주 교통사고 시 알콜농도/data/가해자_음주정도별_주야별_교통사고_20250529134346.xlsx"

# 첫 30행 정도 읽어서 수동 헤더 정의 확인
df_temp = pd.read_excel(file_path, sheet_name="데이터", header=None, nrows=30)

# 실제 데이터 로딩 (수동으로 헤더 지정)
df = pd.read_excel(file_path, sheet_name="데이터", header=None, skiprows=4)

# 수동 헤더 할당
df.columns = ['음주정도', '알콜농도', '주야구분', '사고건수', '사망자수', '부상자수'] + [f'기타_{i}' for i in range(len(df.columns) - 6)]

# 1. '음주운전' 행만 필터링
df = df[df['음주정도'] == '음주운전']

# 2. 정상운전, 측정불능 제거
df = df[~df['알콜농도'].isin(['정상운전', '측정불능'])]

# 3. 숫자형 컬럼 정리
for col in ['사고건수', '사망자수', '부상자수']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4. 결측치 제거
df_clean = df.dropna(subset=['사고건수'])

df_clean.head()


C:\Users\SAMSUNG\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\SAMSUNG\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,음주정도,알콜농도,주야구분,사고건수,사망자수,부상자수,기타_0,기타_1,기타_2,기타_3,...,기타_182,기타_183,기타_184,기타_185,기타_186,기타_187,기타_188,기타_189,기타_190,기타_191


### 혈중알코올농도, 주야별, 연도별, 전체건수

### 결과 파일 저장

In [26]:
import pandas as pd

# 파일 경로
file_path = "C:/Users/SAMSUNG/OneDrive/바탕 화면/workspace/미드프로젝트/데이터 분석/데이터 전처리/음주 교통사고 시 알콜농도/data/가해자_음주정도별_주야별_교통사고_20250529134346.xlsx"

# 멀티헤더 읽기
df_raw = pd.read_excel(file_path, sheet_name="데이터", header=[0, 1, 2])

# 멀티인덱스 컬럼을 단일 문자열로 변환
df_raw.columns = ['_'.join([str(i) for i in col if str(i) != 'nan']).strip() for col in df_raw.columns]

# 컬럼 식별
headers = df_raw.columns.tolist()
col1 = [col for col in headers if '1당음주정도별(1)' in col][0]
col2 = [col for col in headers if '1당음주정도별(2)' in col][0]
col3 = [col for col in headers if '주야별(1)' in col][0]

# '사고건수 (건)' + '전체' 포함된 컬럼만 추출
accident_cols = [col for col in headers if '사고건수' in col and '전체' in col]

# 결측 보간
df_raw[col1] = df_raw[col1].fillna(method='ffill')
df_raw[col2] = df_raw[col2].fillna(method='ffill')
df_raw[col3] = df_raw[col3].fillna(method='ffill')

# 음주운전만 필터링
df_filtered = df_raw[df_raw[col1] == '음주운전']

# 필요한 컬럼 선택
df_selected = df_filtered[[col2, col3] + accident_cols]
df_selected.columns = ['혈중알코올농도', '주야구분'] + accident_cols

# 와이드 → 롱 포맷 변환
df_long = pd.melt(df_selected, 
                  id_vars=['혈중알코올농도', '주야구분'], 
                  var_name='컬럼명', 
                  value_name='사고건수')

# 연도 추출
df_long['연도'] = df_long['컬럼명'].str.extract(r'(\d{4})')

# 숫자형 변환 및 결측 제거
df_long['사고건수'] = pd.to_numeric(df_long['사고건수'], errors='coerce')
df_clean = df_long.dropna(subset=['사고건수'])

# 최종 정리
df_final = df_clean[['혈중알코올농도', '주야구분', '연도', '사고건수']]

# 미리보기
print(df_final.head(20))

# 저장
df_final.to_csv("음주사고_연도별전체사고건수.csv", index=False, encoding='utf-8-sig')
print("\n✅ 연도별 전체 사고건수만 추출하여 저장되었습니다: 음주사고_연도별전체사고건수.csv")

C:\Users\SAMSUNG\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


         혈중알코올농도 주야구분    연도    사고건수
0    0.03~0.049%    주  2019   214.0
1    0.03~0.049%    야  2019   243.0
2   0.05%~0.079%    주  2019   903.0
3   0.05%~0.079%    야  2019  2032.0
4   0.08%~0.149%    주  2019  1501.0
5   0.08%~0.149%    야  2019  5092.0
6   0.15%~0.199%    주  2019   873.0
7   0.15%~0.199%    야  2019  2896.0
8   0.20%~0.249%    주  2019   343.0
9   0.20%~0.249%    야  2019   887.0
10  0.25%~0.299%    주  2019   108.0
11  0.25%~0.299%    야  2019   194.0
12  0.30%~0.349%    주  2019    36.0
13  0.30%~0.349%    야  2019    28.0
14       0.35%이상    주  2019    14.0
15       0.35%이상    야  2019    13.0
16          측정불응    주  2019    93.0
17          측정불응    야  2019   238.0
18   0.03~0.049%    주  2020   365.0
19   0.03~0.049%    야  2020   562.0

✅ 연도별 전체 사고건수만 추출하여 저장되었습니다: 음주사고_연도별전체사고건수.csv


### 주야 구분없이 사고 건수 합치기

In [29]:
import pandas as pd

# CSV 파일 경로
file_path = "음주사고_연도별전체사고건수.csv"

# CSV 불러오기
df = pd.read_csv(file_path)

# 사고건수를 수치형으로 변환 (혹시나 문자열이 있을 경우)
df['사고건수'] = pd.to_numeric(df['사고건수'], errors='coerce')

# 주야구분 없이 같은 연도, 같은 알콜농도별로 사고건수 합산
df_grouped = df.groupby(['연도', '혈중알코올농도'], as_index=False)['사고건수'].sum()


# 미리보기
print(df_grouped.head(20))

# 결과 저장
df_grouped.to_csv('음주사고_연도별_농도별_총사고건수.csv', index=False, encoding='utf-8-sig')
print("\n✅ 연도별 + 혈중알코올농도별로 주야 합산된 사고건수를 저장했습니다.")

      연도       혈중알코올농도    사고건수
0   2019   0.03~0.049%   457.0
1   2019  0.05%~0.079%  2935.0
2   2019  0.08%~0.149%  6593.0
3   2019  0.15%~0.199%  3769.0
4   2019  0.20%~0.249%  1230.0
5   2019  0.25%~0.299%   302.0
6   2019  0.30%~0.349%    64.0
7   2019       0.35%이상    27.0
8   2019          측정불응   331.0
9   2020   0.03~0.049%   927.0
10  2020  0.05%~0.079%  2380.0
11  2020  0.08%~0.149%  7986.0
12  2020  0.15%~0.199%  3997.0
13  2020  0.20%~0.249%  1207.0
14  2020  0.25%~0.299%   227.0
15  2020  0.30%~0.349%    39.0
16  2020       0.35%이상    14.0
17  2020          측정불응   470.0
18  2021   0.03~0.049%   797.0
19  2021  0.05%~0.079%  2103.0

✅ 연도별 + 혈중알코올농도별로 주야 합산된 사고건수를 저장했습니다.


### 사망자수만 구하기

In [19]:
import pandas as pd

# 파일 경로
file_path = "C:/Users/SAMSUNG/OneDrive/바탕 화면/workspace/미드프로젝트/데이터 분석/데이터 전처리/음주 교통사고 시 알콜농도/data/가해자_음주정도별_주야별_교통사고_20250529134346.xlsx"

# 멀티헤더 읽기
df_raw = pd.read_excel(file_path, sheet_name="데이터", header=[0, 1, 2])

# 멀티인덱스 컬럼을 단일 문자열로 변환
df_raw.columns = ['_'.join([str(i) for i in col if str(i) != 'nan']).strip() for col in df_raw.columns]

# 컬럼 식별
headers = df_raw.columns.tolist()
col1 = [col for col in headers if '1당음주정도별(1)' in col][0]
col2 = [col for col in headers if '1당음주정도별(2)' in col][0]
col3 = [col for col in headers if '주야별(1)' in col][0]

# '사망자수 (명)' + '전체' 포함된 컬럼만 추출
accident_cols = [col for col in headers if '사망자수' in col and '전체' in col]

# 결측 보간
df_raw[col1] = df_raw[col1].fillna(method='ffill')
df_raw[col2] = df_raw[col2].fillna(method='ffill')
df_raw[col3] = df_raw[col3].fillna(method='ffill')

# 음주운전만 필터링
df_filtered = df_raw[df_raw[col1] == '음주운전']

# 필요한 컬럼 선택
df_selected = df_filtered[[col2, col3] + accident_cols]
df_selected.columns = ['혈중알코올농도', '주야구분'] + accident_cols

# 와이드 → 롱 포맷 변환
df_long = pd.melt(df_selected, 
                  id_vars=['혈중알코올농도', '주야구분'], 
                  var_name='컬럼명', 
                  value_name='사망자수')

# 연도 추출
df_long['연도'] = df_long['컬럼명'].str.extract(r'(\d{4})')

# 숫자형 변환 및 결측 제거
df_long['사망자수'] = pd.to_numeric(df_long['사망자수'], errors='coerce')
df_clean = df_long.dropna(subset=['사망자수'])

# 최종 정리
df_final = df_clean[['혈중알코올농도', '주야구분', '연도', '사망자수']]

# 주야구분 없이 같은 연도, 같은 알콜농도별로 사고건수 합산
df_final = df_final.groupby(['연도', '혈중알코올농도'], as_index=False)['사망자수'].sum()

# 미리보기
print(df_final.head(20))

# 결과 저장
df_final.to_csv('음주사고_연도별_농도별_총사망자수.csv', index=False, encoding='utf-8-sig')
print("\n✅ 연도별 + 혈중알코올농도별로 주야 합산된 사망자수를 저장했습니다.")

C:\Users\SAMSUNG\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


      연도       혈중알코올농도   사망자수
0   2019   0.03~0.049%   12.0
1   2019  0.05%~0.079%   77.0
2   2019  0.08%~0.149%   73.0
3   2019  0.15%~0.199%   69.0
4   2019  0.20%~0.249%   45.0
5   2019  0.25%~0.299%   14.0
6   2019  0.30%~0.349%    3.0
7   2019       0.35%이상    2.0
8   2019          측정불응    0.0
9   2020   0.03~0.049%   25.0
10  2020  0.05%~0.079%   55.0
11  2020  0.08%~0.149%  102.0
12  2020  0.15%~0.199%   49.0
13  2020  0.20%~0.249%   38.0
14  2020  0.25%~0.299%   13.0
15  2020  0.30%~0.349%    4.0
16  2020       0.35%이상    1.0
17  2020          측정불응    0.0
18  2021   0.03~0.049%   12.0
19  2021  0.05%~0.079%   52.0

✅ 연도별 + 혈중알코올농도별로 주야 합산된 사망자수를 저장했습니다.
